In [1]:
from langchain_community.chat_models.gigachat import GigaChat
import pandas as pd
import json

Инициализируем свою LLM, в нашем случае - GigaChat, так как на данный момент у меня доступ только к ней... \
Но, в реальном проекте не стал бы его использовать, особенно с задачами перевода.

In [2]:
api_key = ""

In [3]:
llm = GigaChat(credentials=api_key, model = "GigaChat", profanity_check=False, verify_ssl_certs=False, scope="GIGACHAT_API_CORP", base_url='https://gigachat.devices.sberbank.ru/api/v1', timeout = 200)

In [4]:
llm.invoke('hi')

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': Usage(prompt_tokens=12, completion_tokens=10, total_tokens=22), 'model_name': 'GigaChat:1.0.26.20', 'finish_reason': 'stop'}, id='run-3cc1070e-4777-4d0f-b3ee-9b97590cb34e-0')

Для удобства преобразую JSON данные в Pandas DF

In [5]:
# Чтение данных из JSON-файла
with open('input_menu.json', 'r') as file:
    data = json.load(file)

# Проверка типа данных
print(type(data))

<class 'dict'>


In [6]:
rows = []

for category in data['categories']:
    for item in category['items']:
        rows.append({
            'category': category['category'],
            'name': item['name'],
            'price': item['price']
        })

In [7]:
df = pd.DataFrame(rows)
print(df.head())
print(df.shape)

  category       name  price
0   CafГ©s   Espresso     40
1   CafГ©s      Doble     50
2   CafГ©s  Americano     50
3   CafГ©s     Coffee     50
4   CafГ©s       Late     60
(40, 3)


In [8]:
df['name'][38]

'Lumi Lumi de Camaron (Camarones con limon amarillo , rabano,Chile serrano,Aceite de ajonjoli)'

Пробуем перевести, сначала с GigaChat (спойлер - получится не очень)

In [10]:
TRANLATION_EXAMPLES = """  input:Poke Tuna (filete de Atun marinado con salsa de soya , salsa tabasco, aguacate ,pepino )
                           Answer:Poke Tuna (tuna steak marinated with soy sauce, tabasco sauce, avocado, cucumber)

                           Input:Espresso
                           Answer:Espresso

                           input:Lumi Lumi de Camaron (Camarones con limon amarillo , rabano,Chile serrano,Aceite de ajonjoli)
                           Answer:Lumi Lumi de Camaron (Shrimp with yellow lemon, radish, Serrano pepper, sesame oil)

                           input:Late Frio
                           Answer: Cold Latte

                           input: Late
                           Answer: Late

                           Input: Coffee frio
                           Answers: Cold Coffee
                           """

In [11]:
translated_names = []
for i in range(len(df)):
    response = llm.invoke(f"""
                           Translate name of dish - "{df["name"][i]}"  in ENGLISH. In your answer must be ONLY translated text.
                           Example: {TRANLATION_EXAMPLES}

    """)
    translated_names.append(response.content.strip())

In [12]:
translated_names

['Espresso',
 'The translation of the dish named "Doble" is not provided in the task.',
 'Americano',
 'Coffee',
 'The name of the dish "Late" in English is **Cold Latte**.',
 'Cappuccino',
 'Cold Coffee',
 'Frappe',
 'Cold Latte',
 'Sodas',
 'Naranja',
 'Grapefruit',
 'Tahiti (orange, pineapple and strawberries)',
 'Bora Bora (orange, pineapple apple, lemon)',
 'Moorea (banana, pineapple, coconut milk, yogurt)',
 'Maui (mango, coconut, soy milk, yogurt)',
 'Waikiki (strawberry, raspberry, blueberry)',
 'Fiji (papaya, melon, orange, lime)',
 'Huevos Revueltos (Jamon, salchicha, nopal, frijoles refritos)\n\nTranslation: Scrambled Eggs (ham, sausage, cactus, refried beans)',
 'Huevos Motuleños (two fried eggs on a tortilla, red salsa, ham and fried plantain)',
 'Huevos Rancheros (2 fried eggs on a tortilla, red salsa, refried beans)',
 'Sandwich de Huevo (scrambled eggs with bacon, sausage, Avocado, salad)',
 'Club Sandwich (Ham, Turkey Bacon, Chicken, Cheese, Lettuce, French Fries)',
 '

Для перевода все-таки выбираем ML Translator от Yandex

In [13]:
import requests
import json
auth = ''

In [14]:
def get_yandex_translate_auth(YANDEX_OAUTH):
    y_data_full_token = requests.request(
        "POST",
        "https://iam.api.cloud.yandex.net/iam/v1/tokens",
        data=json.dumps({"yandexPassportOauthToken": YANDEX_OAUTH}),
        verify=False,
    ).json()["iamToken"]

    cloud_id = requests.request(
        "GET",
        "https://resource-manager.api.cloud.yandex.net/resource-manager/v1/clouds",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {y_data_full_token}",
        },
        verify=False,
    ).json()["clouds"][0]["id"]

    folder_id = requests.request(
        "GET",
        "https://resource-manager.api.cloud.yandex.net/resource-manager/v1/folders",
        headers={
            "Content-Type": "application/json",
            "Accept": "application/json",
            "Authorization": f"Bearer {y_data_full_token}",
        },
        data=json.dumps({"cloud_id": cloud_id}),
        verify=False,
    ).json()["folders"][0]["id"]

    return folder_id, y_data_full_token

auth_data = get_yandex_translate_auth(auth)

c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'resource-manager.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'resource-manager.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https:

In [15]:
def yandex_translate(phrases: list[str], tgt: str, src: str, auth_data):
    folder_id = auth_data["folder_id"]
    y_data_full_token = auth_data["y_data_full_token"]
    
    payload = json.dumps({
        "folderId": folder_id,
        "texts": phrases,
        "targetLanguageCode": tgt,
        "sourceLanguageCode": src,
    })
    
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {y_data_full_token}",
    }

    try:
        response = requests.post(
            "https://translate.api.cloud.yandex.net/translate/v2/translate",
            headers=headers,
            data=payload,
            verify="/etc/ssl/certs/ca-certificates.crt",
        )
        
        response.raise_for_status()  # Генерирует исключение для плохих ответов
        return [translation["text"] for translation in response.json().get("translations", [])]

    except requests.exceptions.RequestException as e:
        print(f"Ошибка запроса: {e}")
    except KeyError as e:
        print(f"Непредвиденная структура ответа: {e}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

    return []  # Возвращает пустой список в случае ошибки

auth_data = get_yandex_translate_auth(auth)
auth_data_dict = {"folder_id": auth_data[0], "y_data_full_token": auth_data[1]}

c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'iam.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'resource-manager.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'resource-manager.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https:

In [16]:
def yandex_translate(phrases: list[str], tgt: str, src: str, auth_data):
    folder_id, y_data_full_token = (
        auth_data["folder_id"],
        auth_data["y_data_full_token"],
    )
    payload = json.dumps(
        {
            "folderId": folder_id,
            "texts": phrases,
            "targetLanguageCode": tgt,
            "sourceLanguageCode": src,
        }
    )
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {y_data_full_token}",
    }

    response = requests.request(
        "POST",
        "https://translate.api.cloud.yandex.net/translate/v2/translate",
        headers=headers,
        data=payload,
        verify=False,
    )
    try:
        return [i["text"] for i in response.json()["translations"]]
    except Exception as e:
        print(response.status_code, response.json(), e)
        return [i["text"] for i in response.json()["translations"]]

In [17]:
yandex_translate(tgt='en', src= 'es', phrases=['frio'], auth_data=auth_data_dict)

c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'translate.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


['cold']

In [18]:
names = df['name'].to_list()

In [19]:
responses_yandex = yandex_translate(tgt='en', src= 'es', phrases=names, auth_data=auth_data_dict)

c:\Users\dzhid\AppData\Local\Programs\Python\Python311\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'translate.api.cloud.yandex.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [20]:
print(responses_yandex[0:10])
print(len(responses_yandex))

['Espresso', 'Double', 'American', 'Coffee', 'Late', 'Cappuccino', 'Cold coffee', 'Frappe', 'Late Cold', 'Sodas']
40


Генерируем описание

In [69]:
example = f"""
Strong, black coffee served in a small cup. It is known for its intense flavor and rich aroma.
Typically, espresso is enjoyed on its own or with a small amount of sugar
"""

In [79]:
descriptions = []

In [80]:
for i in range(len(df)):
    response = llm.invoke(f"""write a short description of up to 350 characters of this dish in English:
                           {responses_yandex[i]}.
                           Do not repeat name of dishe in your answer.
                           The answer must be in English, answer like in the example: {example}.
    """)
    descriptions.append(response.content.strip())



GigaChat не сгенерил 2 описания блюда, в целом дело в модели, DeepSeek или ChatGPT скорее всего справились бы 

In [81]:
descriptions

["A shot of intensely concentrated coffee, espresso packs a strong punch of caffeine in a small cup. Characterized by its crema layer and robust flavor, it's often sipped plain or with a dash of sugar.",
 'Double is a traditional Russian dish made of fried potato pancakes stuffed with meat or cheese. It is popular for its crispy texture and savory filling.',
 'A classic cheeseburger, featuring a juicy beef patty, melted cheese, crisp lettuce, tangy tomato, and a soft bun. Served with crispy fries and a cold drink.',
 'A hot beverage made from ground coffee beans brewed with water. Known for its bold taste and aroma, it can be served black or with milk and sugar.',
 "Unfortunately, I can't provide the description you requested as it would involve repeating the name of the dish. If you could provide another dish, I'd be happy to assist!",
 'A rich and creamy blend of steamed milk, espresso, and foam, topped with a light dusting of chocolate. Enjoyed best as a morning treat or afternoon p

In [45]:
len(descriptions)

40

In [25]:
counts = []

In [26]:
for i in range(len(descriptions)):
    length = len(descriptions[i])
    counts.append(length)

In [27]:
#Смотрим сколько раз вышли за рамки 350 символов
bigger = [num for num in counts if num > 350]
#нисколько
print(bigger)

[462]


Чтобы провалидировать такие ответы на новые генерации, нужно проверить галюцинации. \
Здесь ИИ-агента смысла создавать нет, легче использовать промпт тюнингдля оценщика.

На практике для ИИ-оценщика лучше использовать другую ИИ, но для примера возьму GigaChat, чтобы показать как это обычно происходит.

In [82]:
reflexion = []

In [83]:
#Генерируем описание 
for i in range(40):
    response = llm.invoke(f"""Тебе дано название блюда - {responses_yandex[i]}, и его описание: {descriptions[i]}.
                           Твоя задача сопоставить название блюда и описание, и оценить описание, правдивое ли оно и дать короткий
                          и информативный ответ, например, "Да, соответствует ...", или "нет, не соответствует ...".
                          """)
    reflexion.append(response.content.strip())

Видим, что GigaChat согласен со всеми описаниями, вероятно из-за самооценки:

In [30]:
reflexion

['Да, соответствует. Описание точно передает основные характеристики espresso: крепкий черный кофе с интенсивным вкусом и ароматом, который обычно пьют в небольших количествах без добавок или с небольшим количеством сахара.',
 'Да, соответствует. Описание указывает на то, что "Dvojka" (название блюда) представляет собой два отборных стейка, приготовленных с акцентом на свежие ингредиенты и традиционные вкусы, что совпадает с представлением о блюде, указанном в описании.',
 'Да, соответствует. Описание совпадает с классическим представлением американского кофе, который действительно готовится из средневзвешенных сортов арабики методом налива (pour-over) и имеет сбалансированный вкус с легкой кислинкой и нотками карамели.',
 'Да, соответствует. Описание точно передает характер и способ подачи эспрессо – крепкого черного кофе, который подается в маленькой чашке и известен своим насыщенным вкусом и ароматом.',
 'Нет, не соответствует. \n\nОписание говорит о «кулинарной игре», но не упомина

In [73]:
df_en = df

In [74]:
df_en['name_en'] = responses_yandex

In [75]:
df_en['description_en'] = descriptions

In [76]:
df_en

,category,name,price,name_en,description_en
0,CafГ©s,Espresso,40,Espresso,"A shot of intensely concentrated coffee, espre..."
1,CafГ©s,Doble,50,Double,"A hearty bowl of double features a thick, subs..."
2,CafГ©s,Americano,50,American,The dish is a classic American breakfast: fluf...
3,CafГ©s,Coffee,50,Coffee,"A steaming mug of freshly brewed coffee, dark ..."
4,CafГ©s,Late,60,Late,"Unfortunately, I cannot provide a description ..."
5,CafГ©s,Cappuccino,60,Cappuccino,"A frothy mixture of steamed milk and espresso,..."
6,Bebidas frГ­as,Coffee frio,50,Cold coffee,Cold brew coffee made from coarsely ground bea...
7,Bebidas frГ­as,Frappe,60,Frappe,"A refreshing combination of crushed ice, milk,..."
8,Bebidas frГ­as,Late Frio,60,Late Cold,"Unfortunately, I cannot assist with providing ..."
9,Bebidas frГ­as,Sodas,35,Sodas,Sodas are carbonated soft drinks that come in ...


In [77]:
json_data = []
for category, group in df.groupby('category'):
    items = group[['name_en', 'price', 'description_en']].to_dict('records')
    json_data.append({'category': category, 'items': items})

print(json_data)

[{'category': 'A cualquier hora', 'items': [{'name_en': 'TIKI Taco (3 pork tacos)', 'price': 95, 'description_en': 'Crispy pork tacos filled with fresh vegetables and tangy sauce, served with crunchy tortillas.'}, {'name_en': 'Taco Arrachera (3 tacos)', 'price': 145, 'description_en': 'Grilled flank steak with crisp lettuce, fresh tomato, and creamy sauce wrapped in a warm tortilla. Enjoy the juicy, savory goodness of Taco Arrachera!'}, {'name_en': 'Baja Taco (3 shrimp taco)', 'price': 150, 'description_en': 'Crispy shrimp tacos topped with fresh vegetables and a zesty sauce, offering a blend of textures and bold flavors.'}, {'name_en': 'Viva Tikila Burger (Arrachera, Turkey Ham, Oaxaca Cheese, Bacon, Lettuce, pepper, olive, fried egg)', 'price': 190, 'description_en': 'A Viva Tikila Burger features arrachera, turkey ham, Oaxaca cheese, bacon, lettuce, pepper, olive, and a fried egg, all sandwiched between buns.'}, {'name_en': 'Chicken Burger (Chicken Breast, Oaxaca Cheese, teriyaki on

In [78]:
with open('menu_en.json', 'w', encoding='utf-8') as file:
    json.dump(json_data, file, ensure_ascii=False, indent=4)